In [2]:
import numpy as np
import tensorflow as tf
from scipy.optimize import linprog

In [3]:
def simulate_training_data(num_samples=10000, num_features=100, skewness=2.0):
    # Generate categorical features with skewed distributions
    feature_counts = np.random.zipf(skewness, size=num_features) # Assume we use the random skewed data
    feature_counts = feature_counts / feature_counts.sum()

    # Generate samples
    samples = np.random.choice(num_features, size=(num_samples, 1), p=feature_counts)

    return samples

# Simulate data
training_data = simulate_training_data()

In [5]:
def recshard(num_features, hbm_capacity, dram_capacity):
    # Objective: Minimize the use of slower DRAM
    c = np.append(np.zeros(hbm_capacity), np.ones(dram_capacity))

    # Constraints: Sum of features in HBM and DRAM should be equal to total features
    A_eq = np.append(np.ones(hbm_capacity), np.ones(dram_capacity)).reshape(1, -1)
    b_eq = [num_features]

    # Bounds: Features can only be assigned to HBM or DRAM
    bounds = [(0, 1)] * (hbm_capacity + dram_capacity)

    # Solve the MILP problem
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    # Extract the optimal solution
    hbm_allocation = result.x[:hbm_capacity]
    dram_allocation = result.x[hbm_capacity:]

    return hbm_allocation, dram_allocation

# Example usage
hbm_capacity = 10  # Number of features that can fit in HBM
dram_capacity = 20  # Number of features that can fit in DRAM
hbm_allocation, dram_allocation = recshard(num_features=30, hbm_capacity=hbm_capacity, dram_capacity=dram_capacity)
print(f"HBM Allocation: {hbm_allocation}")
print(f"DRAM Allocation: {dram_allocation}")

HBM Allocation: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
DRAM Allocation: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
def flexshard(num_features, num_nodes, node_capacity):
    # Simulate feature usage across nodes
    feature_usage = np.random.randint(0, 100, size=(num_features, num_nodes))

    # Calculate the total usage per node
    total_usage = feature_usage.sum(axis=0)

    # Sort features by their total usage
    sorted_indices = np.argsort(total_usage)

    # Distribute features to nodes
    node_allocation = np.zeros((num_features, num_nodes))
    for i, feature in enumerate(range(num_features)):
        # Assign feature to the node with the least total usage
        node = sorted_indices[i % num_nodes]
        node_allocation[feature, node] = 1
        total_usage[node] += feature_usage[feature, node]

    return node_allocation

# Example usage
num_nodes = 4
node_capacity = 10
node_allocation = flexshard(num_features=40, num_nodes=num_nodes, node_capacity=node_capacity)
print(f"Node Allocation:\n{node_allocation}")

Node Allocation:
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


The value 1 indicates that a feature is assigned to a particular node, while 0 isn't.

Here more clearly breakdown
- Feature 0: Assigned to Node 3 ([0. 0. 0. 1.])
- Feature 1: Assigned to Node 0 ([1. 0. 0. 0.])
- Feature 2: Assigned to Node 2 ([0. 0. 1. 0.])
- Feature 3: Assigned to Node 1 ([0. 1. 0. 0.])
- Feature 4: Assigned to Node 3 ([0. 0. 0. 1.])
- Feature 5: Assigned to Node 0 ([1. 0. 0. 0.])
- And so on ...

As you look, the pattern shows that features are being evenly distributed accros the nodes
in round-robin fashion. This's because the "flexshard" function assigns features to nodes
in a cyclic manner based on the least total usage, which, in this simplified example,
results in a round robin distribution. (Hopefully you can implement this in the deep learning)



Implications
- Balance Load, ensuring no single node becomes bottleneck
- By distributing features evenly, the inter-node communication is minimized